In [35]:
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torchvision as tv
import torchvision.transforms as transforms
import sys, os
from sklearn.model_selection import train_test_split as tts
from scipy.ndimage import uniform_filter1d
from torch.utils.data import Dataset, DataLoader

sys.path.append(os.getcwd())
import models


In [36]:
# Import the keys
keys = list(np.load("keys/all_keys.npy"))
keys = [tuple(k) for k in keys]

# Import the data and labels
mi_matrices = {
    key:np.load(f"MI_matrices/MI_matrix_{key}.npy") for key in keys
}
data = {
    key:np.load(f"data/data_{key}.npy") for key in keys
}

# Sort data into a list, with the same ordering as the MI_matrices
mi_matrices_list = list()
data_list = list()
for key in keys:
    mi_matrices_list.append(mi_matrices[key])
    data_list.append(data[key])

In [ ]:
# Hyperparams
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
batch_size=12
learning_rate = 0.01
num_epochs = 5
input_size = 5000

In [39]:
# Numpify the data
data = np.asarray(data_list).reshape((48, 5000, 11))
labels = np.array(mi_matrices_list).reshape((48, 11, 11))

# Split the data into train and test
data_train, data_test, label_train, label_test = tts(data, labels, test_size=0.1)

# Modify variable type to tensor
data_train = models.NumbersDataset(data_train, label_train)
data_test = models.NumbersDataset(data_test, label_test)

# Create the dataloaders
loader_train = DataLoader(data_train, batch_size=batch_size, shuffle=True)
loader_test = DataLoader(data_test, batch_size=batch_size, shuffle=True)